# Introduction

In my last notebook I finetuned a pretrained model for the purpose of translation and summarization. It was a long and resource intensive process while the results were less than stellar. Even the word pairs used to train the model had many mistakes. It would be much more efficient to instead use openAI's GPT 4 for similar small scale tasks. 

Despite many of the advances since GPT-3.5, GPT-4 still ocasionally struggles with basic reading comprehension and arithmetmic.

In this notebook I am going to explore the strengths and limitations as well as how to get the best out of the GPT-4 model using various prompt engineering techniques.

## Load the Libraries and API Key

In [ ]:
!pip install openai

In [14]:
from openai import OpenAI

client = OpenAI(api_key = 'YOUR KEY HERE')

## Create a Function for Prompting

This function will be called with a prompt and output printed for each instance.

In [41]:
def get_completion(prompt, model='gpt-4',temperature=0, max_tokens=500):
    messages =[{'role':'user', 'content':prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

Let's ask it a basic question just to make sure that everything is working.

In [22]:
prompt=get_completion('what is two plus two')
print(prompt)

Two plus two equals four.


# Prompt Optimization

## Combining Letters

One of GPT 4's biggest issues is with counting individiual letters within words. That is because it percieves information in tokens rather that letters or words which are often different.

In [29]:
prompt=get_completion(
"""
Concatenate the second letter in these words together:
Egregious
Assistant
Compupter
"""
)
print(prompt)

rgu


For example, deciding which letter in a word is the second and combining these letters together can be difficult. One of the simplest methods to fix mistakes like this is to tell ChatGPT to reread the words. Sometimes simply prompting a few times will output the correct answer but in situations where errors should be compltely avoided it is better to have a method to deal with these shortcomings.

In [30]:
prompt=get_completion(
"""
Concatenate the second letter in these words together:
Egregious
Assistant
Compupter

Reread all of these words again and make sure to concatenate the second letter in these words together.
"""
)
print(prompt)

The second letter in each word is:
Egregious - g
Assistant - s
Computer - o

When concatenated together, they form "gso".


Let's take a look at another example since it a big issue for the model.

In [78]:
prompt=get_completion(
"""
Concatenate the first and last letters of the word Salt the second and third letters of the word Useful and the last two letters \
of the word Table
"""
)
print(prompt)

The result is "Stusle".


In [82]:
prompt=get_completion(
"""
Concatenate the first and last letters of the word Salt the second and third letters of the word Useful and the last two letters \
of the word Table.

Write out how you would concatenate the letters from these words in python. Remember that Python starts counting at 0.
What would be the output?
"""
)
print(prompt)

In Python, you can concatenate the letters from these words using the following code:

```python
word1 = "Salt"
word2 = "Useful"
word3 = "Table"

output = word1[0] + word1[-1] + word2[1] + word2[2] + word3[-2] + word3[-1]
print(output)
```

The output would be: "Stsele"


Another technique is to prompt the model to think in terms of code which is a more natural logical process than being confused with words and tokens. Even in this example I had to include a reminder about how counting in Python works.

## Math Word Problem

Math itself is usually not a problem but when words and multiple steps are added, it can easily cause the model to skip a step or focus on a different variable.

In [35]:
prompt=get_completion(
"""
Bob has five apples and five oranges. Alice gave him three apples and took five oranges. 
The next day he ate half of his apples for breakfast and then another one for lunch. Later that day he bought and orange, an apple
and two bananas. Bob gave all of his apples to alice. How many oranges does he have now?
"""
)
print(prompt)

Bob has 2 oranges now.


In [42]:
prompt=get_completion(
"""
Bob has five apples and five oranges. Alice gave him three apples and took five oranges. 
The next day he ate half of his apples for breakfast and then another one for lunch. Later that day he bought and orange, an apple
and two bananas. Bob gave all of his apples to alice. How many oranges does he have now?

Break down everything that happened to Bob step by step, making sure to count the total number of each fruit he has at each step.
How many oranges does Bob have now?
"""
)
print(prompt)

Step 1: Bob starts with five apples and five oranges.

Step 2: Alice gives Bob three apples and takes five oranges. Now, Bob has eight apples (5 original + 3 from Alice) and zero oranges (5 original - 5 taken by Alice).

Step 3: The next day, Bob eats half of his apples for breakfast. Half of eight apples is four, so Bob now has four apples left.

Step 4: Bob eats another apple for lunch. Now, he has three apples left.

Step 5: Later that day, Bob buys an orange, an apple, and two bananas. Now, Bob has four apples (3 remaining + 1 bought) and one orange (0 original + 1 bought).

Step 6: Bob gives all of his apples to Alice. Now, Bob has zero apples and one orange.

So, Bob now has one orange.


Telling the model to break something down step by step forces it to break each step down into more detail and 'think' it through making sure that no step is missed or confused with another.

## Telling and Counting Time

Terms like half an hour, quarter to something or ten past something can be confusing for the model

In [43]:
prompt=get_completion(
"""
Bob has an important meeting he has to attend at eight. He left his house at quarter to seven and headed to the bus stop. /
By the time he got to the bus stop it was already five past seven. He missed the first bus and had to wait another 22 minutes for the next one./
The bus ride took him half an hour. How much time does Bob have until the meeting?
"""
)
print(prompt)

Bob has 15 minutes until the meeting.


In [44]:
prompt=get_completion(
"""
Q:
Alice has an important meeting at five. She left her house at ten past three and got into her car.
She decided to stop by the gas station to refuel her car. By the time she was done it was already quarter to four.
She drove for 48 minutes. How much time does Alice have until the meeting?


A:
Left the house at ten past three or 3:10
Got gas at quarter to four or 3:45
Drove for 48 minutes so 3:45 + 48 = 4:33
The meeting is at 5:00 - 4:33 = 27
Alice has 27 minutes until the meeting

Q:
Bob has an important meeting he has to attend at eight. He left his house at quarter to seven and headed to the bus stop. 
By the time he got to the bus stop it was already five past seven. He missed the first bus and had to wait another 22 minutes for the next one.
The bus ride took him half an hour. How much time does Bob have until the meeting?

A:
"""
)
print(prompt)

Left the house at quarter to seven or 6:45
Got to the bus stop at five past seven or 7:05
Waited for the bus for 22 minutes so 7:05 + 22 = 7:27
The bus ride took half an hour so 7:27 + 30 = 7:57
The meeting is at 8:00 - 7:57 = 3
Bob has 3 minutes until the meeting.


One of the most powerful techniques in prompt engineering is to first give an example on how to best solve a similar problem. GPT 4 will then break it down in a similar way and give the correct answer.

## Multilingual Prompting and Translation

Let's take a look at another word problem and see how translation can be used to improve accuracy.

In [94]:
prompt=get_completion(
"""
Alice began her journey from England and decided to see how many fridge magnets she can collect on the way. First she went \
to Norway where she got two magnets. She then got a magnet in Sweden, Denmark and Finland. In Germany she was able to get \
half a dozen but only a single one in Switzerland. In Italy she bought three but lost one. In Spain she was able to get three of \
them. Just one in Portugal but France did not disappoint with an impressive five magnets. She couldn't wait to get home and \
place them all on her fridge.

How many magnets will Alice bring home?
"""
)
print(prompt)

Alice will bring home 20 magnets.


In [95]:
prompt=get_completion(
"""
Alice began her journey from England and decided to see how many fridge magnets she can collect on the way. First she went \
to Norway where she got two magnets. She then got a magnet in Sweden, Denmark and Finland. In Germany she was able to get \
half a dozen but only a single one in Switzerland. In Italy she bought three but lost one. In Spain she was able to get three of \
them. Just one in Portugal but France did not disappoint with an impressive five magnets. She couldn't wait to get home and \
place them all on her fridge.

Translate the above to Russian.
"""
)
print(prompt)

Алиса начала свое путешествие из Англии и решила посмотреть, сколько магнитов на холодильник она сможет собрать по пути. Сначала она поехала в Норвегию, где получила два магнита. Затем она получила по магниту в Швеции, Дании и Финляндии. В Германии ей удалось получить полдюжины, но только один в Швейцарии. В Италии она купила три, но потеряла один. В Испании она смогла получить три из них. Только один в Португалии, но Франция не разочаровала с впечатляющими пятью магнитами. Она не могла дождаться, когда вернется домой и разместит их все на своем холодильнике.


In [122]:
prompt=get_completion(
"""
Алиса начала свое путешествие из Англии и решила посмотреть, сколько магнитов на холодильник она сможет собрать по пути. \
Сначала она поехала в Норвегию, где получила два магнита. Затем она получила по магниту в Швеции, Дании и Финляндии. \
В Германии ей удалось получить полдюжины, но только один в Швейцарии. В Италии она купила три, но потеряла один. \
В Испании она смогла получить три из них. Только один в Португалии, но Франция не разочаровала с впечатляющими пятью магнитами. \
Она не могла дождаться, когда вернется домой и разместит их все на своем холодильнике.

Подведи Итог.
"""
)
print(prompt)

Алиса собрала в общей сложности 2 (Норвегия) + 1 (Швеция) + 1 (Дания) + 1 (Финляндия) + 6 (Германия) + 1 (Швейцария) + 3 (Италия) - 1 (потерянный в Италии) + 3 (Испания) + 1 (Португалия) + 5 (Франция) = 23 магнита.


The model had issues following along with the number of magnets in this word problem. I decided to translate the problem to Russian and then ask it to draw a conclusion from the translation. One interesting thing here is that translating to a different language can clear up some ambiguities. For example in English some can misunderstand whether there was one magnet from Sweden, Denmark and Finland or one from each. The Russian translation is much less ambigous that there is a magnet from each country. Alternatively, simply giving the model an opportunity to re read and summarize the problem could help it arrive to the correct conclusion. 

In [123]:
prompt=get_completion(
"""
С вами на вы или на ты?
"""
)
print(prompt)

Как вам будет удобнее! Я могу общаться как в формальном, так и в неформальном стиле.


I was worried that I might have offended the model by using the informal you, but luckily both the formal and the informal are fine!

# Conclusion

Despite the technological breakthroughs that lead to the creation of this model, knowing that it can still struggle with questions that an elementary school student can answer shines a light on its limitiations. The knowledge of how to circumvent these issues at the current stage of development is very valuable. I hope to monitor the model's progress closely and learn together with it.